# EDSA Movie Recommendation challenge

<img src="https://media.giphy.com/media/3ohhwDMC187JqL69DG/giphy.gif">

# Introduction

In Today's technology driven world,recommender systems are socially and economically critical for ensuring that idividuals can make appropriate choices surrounding the content they engage wit on a daily basis. One application where this is especially true surrounds movie content recommendations;where intelligent algorithms can help viewers find great titles from tens of thousands of movies

With this context,we were able to construct a recommendation algorithm based on content or collaborative filtering,cabable of accurately predicting how a user will rate a movie theyhave not  yet viewed based on their historical preferences.

Providing an accurate and robust solution to this challenge has immense economic potential,with users of the system being exposed to the content they would like to view or purchase- generating revenue and platform affinity

Authors:Brighton Nkomo,Claudia Elliot-Wilson,Eve Sithhole,Reneilwe Mphahlele,Vuyiswa M

# Table of contents

1. Install packages
2. Reading in data
3. Data Pre-processing
4. Exploratory Data Analysis
5. Feature Engineering
6. Model Selection
7. Submission
8. Conclusion

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Installing packages
Please download all relevant packages in. There is no terminal so you will pip install everything.

You can find a list of recommended install from the Intro to Recommender sysytem notebook.

In [ ]:
# Install packages here
# Packages for data processing
import numpy as np
import pandas as pd
import datetime
from sklearn import preprocessing
from sklearn.datasets import make_blobs
from sklearn.preprocessing import StandardScaler
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import re
from scipy.sparse import csr_matrix
import scipy as sp

# Packages for visualization
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
from wordcloud import WordCloud, STOPWORDS

# Packages for modeling
from surprise import Reader
from surprise import Dataset
from surprise import KNNWithMeans
from surprise import KNNBasic
from surprise.model_selection import cross_validate
from surprise.model_selection import GridSearchCV
from surprise import SVD
from surprise import SVDpp
from surprise import NMF
from surprise import SlopeOne
from surprise import CoClustering
from surprise import NormalPredictor, BaselineOnly
import heapq

# Packages for model evaluation
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from time import time

# Package to suppress warnings
import warnings
warnings.filterwarnings("ignore")

# Packages for saving models
import pickle

# Reading in data

In [ ]:
df_sample_submission = pd.read_csv('sample_submission.csv')
df_movies = pd.read_csv('movies.csv')
df_imdb = pd.read_csv('imdb_data.csv')
df_genome_scores = pd.read_csv('genome_scores.csv')
df_genome_tags = pd.read_csv('genome_tags.csv')
df_train = pd.read_csv('train.csv')
df_test = pd.read_csv('test.csv')
df_tags = pd.read_csv('tags.csv')
df_links = pd.read_csv('links.csv')

# Data Pre-processing

In [ ]:
#Using regular expressions to find a year stored between parentheses
#We specify the parantheses so we don't conflict with movies that have years in their titles
df_movies['year'] = df_movies.title.str.extract('(\(\d\d\d\d\))',expand=False)
#Removing the parentheses
df_movies['year'] = df_movies.year.str.extract('(\d\d\d\d)',expand=False)
#Removing the years from the 'title' column
df_movies['title'] = df_movies.title.str.replace('(\(\d\d\d\d\))', '')
#Applying the strip function to get rid of any ending whitespace characters that may have appeared
df_movies['title'] = df_movies['title'].apply(lambda x: x.strip())

In [ ]:
#Every genre is separated by a | so we simply have to call the split function on |
df_movies['genres'] = df_movies.genres.str.split('|')
df_movies.head()

# Exploratory Data Analysis

### Title Wordcloud

Are there certain words that figure more often in Movie Titles? There are some words which are considered more potent and considered more worthy of a title. Let us find out!

In [ ]:
df_movies['title'] = df_movies['title'].astype('str')
title_corpus = ' '.join(df_movies['title'])

In [ ]:
title_wordcloud = WordCloud(stopwords=STOPWORDS, background_color='white', height=2000, width=4000).generate(title_corpus)
plt.figure(figsize=(16,8))
plt.imshow(title_wordcloud)
plt.axis('off')
plt.show()

The word Love is the most commonly used word in movie titles. Girl, Day and Man are also among the most commonly occuring words. We think this encloses the idea of the universal presence of romance in movies pretty well.

### Numbers of movies by year

The dataset available to us does not present an entire collection of movies released since the inception of cinema. However, it is reasonable to assume that it does include almost every major film released in Hollywood as well as other major film industries across the world (such as Bollywood in India). With this assumption in mind, let us take a look at the number of movies produced by the year.

In [ ]:
#checking numbers of movies by year
year_count = df_movies.groupby('year')['title'].count()
plt.figure(figsize=(18,5))
year_count.plot()

We notice that there is a sharp rise in the number of movies starting the 1990s decade. However, we will not look too much into this as it is entirely possible that recent movies were oversampled for the purposes of this dataset.

Next, let us take a look at the earliest movies represented in the dataset.

In [ ]:
df_movies[df_movies['year'] != 'NaT'][['title', 'year']].sort_values('year').head(10)

The oldest movie, Passage of Venus, was a series of photographs of the transit of the planet Venus across the Sun in 1874. This shows how the movies industry has evolved over the year to what it is now.

### Runtime

In [ ]:
df_imdb['runtime'].describe()

The average length of a movie is about 1 hour and 30 minutes. The longest movie on record in this dataset is a staggering 877 minutes (15 hours) long.

We are aware that most movies(mainstream movies) are less than 5 hours (or 300 minutes) long.Below we plottin a distribution of these mainstream movies

In [ ]:
#plot showing distribution of mainstream movies
df_imdb['runtime'] = df_imdb['runtime'].astype('float')
plt.figure(figsize=(12,6))
sns.distplot(df_imdb[(df_imdb['runtime'] < 300) & (df_imdb['runtime'] > 0)]['runtime'])

### Most common Genres

Using the df_movies data, which includes information on the genres each movie is categorised into, we can visualise which genres have the greatest number of movies. While this is not necessarily reflective of which genres are the most popular, it can give us an idea of what is contained within our data.

In [ ]:
# Create dataframe containing only the movieId and genres
movies_genres = pd.DataFrame(df_movies[['movieId', 'genres']],
                             columns=['movieId', 'genres'])

# Split genres seperated by "|" and create a list containing the genres allocated to each movie
#movies_genres.genres = movies_genres.genres.apply(lambda x: x.split('|'))

# Create expanded dataframe where each movie-genre combination is in a seperate row
movies_genres = pd.DataFrame([(tup.movieId, d) for tup in movies_genres.itertuples() for d in tup.genres],
                             columns=['movieId', 'genres'])

movies_genres.head()

In [ ]:
# Plot the genres from most common to least common
plot = plt.figure(figsize=(15, 10))
plt.title('Most common genres\n', fontsize=20)
sns.countplot(y="genres", data=movies_genres,
              order=movies_genres['genres'].value_counts(ascending=False).index,
              palette='Reds_r')
plt.show()

The graph shows us that a majority of the movies in this database (over 25000!) are listed under the genre "Drama", and the least under the genre "IMAX". We can also see that there are quite a few movies, around 5000, that have no genres listed for them.

### Ratings Distribution

We can also explore the distribution of ratings across the data, to see which ratings are given most often, and which least often.

In [ ]:
ratings = pd.DataFrame(df_train[['rating']],
                             columns=['rating'])

ratings.head()

In [ ]:
plt.figure(figsize=(15, 10))
plt.title('Ratings distribution \n', fontsize=20)
sns.countplot(x="rating", data=ratings,
              palette='Reds_r')
plt.show()

The countplot shows more movies are given ratings of 4.0 and 3.0, and fewer movies are given ratings of 0.5 and 1.5. This indicates that the movies in this dataset are often given average to good ratings, with fewer movies being rated poorly.

### Ratings per Movie

Another aspect of the data we can explore is how many ratings there are per movie. As there are many movies in the dataset, how many of these actually have ratings from a lot of users?

In [ ]:
#Because there are so many movies in the database, we clip it at 100, otherwise the graph becomes too difficult to read
ratings_per_movie = pd.DataFrame(df_train.groupby('movieId')['rating'].count().clip(upper=100))
ratings_per_movie.groupby('rating')['rating'].count()

In [ ]:
plt.figure(figsize=(15, 10))
plt.title('Ratings per Movie \n', fontsize=20)
sns.histplot(data=ratings_per_movie,
              palette='Reds_r')
plt.show()

The histogram shows us that a vast majority of the movies (12537 movies) only have one rating. We can see that as the number of ratings per movie increases, the count of movies decreases, indicating that few movies have been rated by a large number of users.

### Ratings per User

On the other hand, we can also explore how many users are actually giving ratings to movies.

In [ ]:
#Again, clipped at 500 because there are so many users, that including them all would make the graph difficult to read
ratings_per_user = pd.DataFrame(df_train.groupby('userId')['rating'].count().clip(upper=500))
ratings_per_user.groupby('rating')['rating'].count()

In [ ]:
plt.figure(figsize=(15, 10))
plt.title('Ratings per User \n', fontsize=20)
sns.histplot(data=ratings_per_user,
              palette='Reds_r')
plt.show()

Similarly to the previous graph, we can see that the count of users dwindles as the number of ratings per user increases. Quite a lot of users appear to have given between 1 and 100 ratings, but increasingly fewer have given more ratings than this.

# Data-Preparation

Since keeping genres in a list format isn't optimal for the content-based recommendation system technique, we will use the One Hot Encoding technique to convert the list of genres to a vector where each column corresponds to one possible value of the feature. This encoding is needed for feeding categorical data. In this case, we store every different genre in columns that contain either 1 or 0. 1 shows that a movie has that genre and 0 shows that it doesn't. Let's also store this dataframe in another variable since genres won't be important for our first recommendation system.

In [ ]:
#Copying the movie dataframe into a new one since we won't need to use the genre information in our first case.
moviesWithGenres_df = df_movies.copy()

#For every row in the dataframe, iterate through the list of genres and place a 1 into the corresponding column
for index, row in df_movies.iterrows():
    for genre in row['genres']:
        moviesWithGenres_df.at[index, genre] = 1
#Filling in the NaN values with 0 to show that a movie doesn't have that column's genre
moviesWithGenres_df = moviesWithGenres_df.fillna(0)
moviesWithGenres_df.head()

Now let's look at the ratings.

In [ ]:
df_train.head()

Every row in the df_train dataframe has a user id associated with at least one movie. Here, we do not need the timestamp of when the users reviewed the movie. So, let's drop the 'timestamp' column.

In [ ]:
df_train = df_train.drop('timestamp', 1)

In [ ]:
df_train.head()

# Content-Based Recomendation System

Let's begin by creating an input user to recommend movies to:

Notice: To add more movies, simply increase the amount of elements in the userInput. Feel free to add more in! Just be sure to write it in with capital letters and if a movie starts with a "The", like "The Matrix" then write it in like this: 'Matrix, The' .

In [ ]:
userInput = [
            {'title':'Breakfast Club, The', 'rating':5},
            {'title':'Toy Story', 'rating':3.5},
            {'title':'Jumanji', 'rating':2},
            {'title':"Pulp Fiction", 'rating':5},
            {'title':'Akira', 'rating':4.5}
         ] 
inputMovies = pd.DataFrame(userInput)
inputMovies

In [ ]:
#Filtering out the movies by title
inputId = df_movies[df_movies['title'].isin(inputMovies['title'].tolist())]
#Then merging it so we can get the movieId. It's implicitly merging it by title.
inputMovies = pd.merge(inputId, inputMovies)
#Dropping information we won't use from the input dataframe
inputMovies = inputMovies.drop('genres', 1).drop('year', 1)
#Final input dataframe
#If a movie you added in above isn't here, then it might not be in the original 
#dataframe or it might spelled differently, please check capitalisation.
inputMovies

In [ ]:
#Filtering out the movies from the input
userMovies = moviesWithGenres_df[moviesWithGenres_df['movieId'].isin(inputMovies['movieId'].tolist())]
userMovies

We'll only need the actual genre table, so let's clean this up a bit by resetting the index and dropping the movieId, title, genres and year columns.

In [ ]:
#Resetting the index to avoid future issues
userMovies = userMovies.reset_index(drop=True)
#Dropping unnecessary issues due to save memory and to avoid issues
userGenreTable = userMovies.drop('movieId', 1).drop('title', 1).drop('genres', 1).drop('year', 1)
userGenreTable

Now we're ready to start learning the input's preferences!

To do this, we're going to turn each genre into weights. We can do this by using the input's reviews and multiplying them into the input's genre table and then summing up the resulting table by column. This operation is actually a dot product between a matrix and a vector, so we can simply accomplish by calling Pandas's "dot" function.

In [ ]:
inputMovies['rating']

In [ ]:
#Dot produt to get weights
userProfile = userGenreTable.transpose().dot(inputMovies['rating'])
#The user profile
userProfile

Now, we have the weights for every of the user's preferences. This is known as the User Profile. Using this, we can recommend movies that satisfy the user's preferences.

Let's start by extracting the genre table from the original dataframe:

In [ ]:
#Now let's get the genres of every movie in our original dataframe
genreTable = moviesWithGenres_df.set_index(moviesWithGenres_df['movieId'])
#And drop the unnecessary information
genreTable = genreTable.drop('movieId', 1).drop('title', 1).drop('genres', 1).drop('year', 1)
genreTable.head()

In [ ]:
genreTable.shape

With the input's profile and the complete list of movies and their genres in hand, we're going to take the weighted average of every movie based on the input profile and recommend the top twenty movies that most satisfy it.

In [ ]:
#Multiply the genres by the weights and then take the weighted average
recommendationTable_df = ((genreTable*userProfile).sum(axis=1))/(userProfile.sum())
recommendationTable_df.head()

In [ ]:
#Sort our recommendations in descending order
recommendationTable_df = recommendationTable_df.sort_values(ascending=False)
#Just a peek at the values
recommendationTable_df.head()

Now here's the recommendation table!

In [ ]:
#The final recommendation table
df_movies.loc[df_movies['movieId'].isin(recommendationTable_df.head(20).keys())]

Although content-based recommender systems are highly personalized for the user and they learn the user's preferences, one major disadvantage of them is that they don't take into account of what others may think of the item. So low quality recommendations may happen. This is where collaborative filtering steps in to resolve this problem.

# Collaborative Filtering

Let's also drop the genres column since we won't need it for this particular recommendation system.

In [ ]:
#Dropping the genres column
df_movies = df_movies.drop('genres', 1)

In [ ]:
df_movies.head()

Now, time to start our work on recommendation systems. 

The first technique we're going to take a look at is called __Collaborative Filtering__, which is also known as __User-User Filtering__. As hinted by its alternate name, this technique uses other users to recommend items to the input user. It attempts to find users that have similar preferences and opinions as the input and then recommends items that they have liked to the input. There are several methods of finding similar users (Even some making use of Machine Learning), and the one we will be using here is going to be based on the __Pearson Correlation Function__.

<img src="https://ibm.box.com/shared/static/1ql8cbwhtkmbr6nge5e706ikzm5mua5w.png" width=800px>


The process for creating a User Based recommendation system is as follows:
- Select a user with the movies the user has watched
- Based on his rating to movies, find the top X neighbours 
- Get the watched movie record of the user for each neighbour.
- Calculate a similarity score using some formula
- Recommend the items with the highest score

#### The users who has seen the same movies
Now with the movie ID's in our input, we can now get the subset of users that have watched and reviewed the movies in our input.

In [ ]:
#Filtering out users that have watched movies that the input has watched and storing it
userSubset = df_train[df_train['movieId'].isin(inputMovies['movieId'].tolist())]
userSubset.head()

We now group up the rows by user ID.

In [ ]:
#Groupby creates several sub dataframes where they all have the same value in the column specified as the parameter
userSubsetGroup = userSubset.groupby(['userId'])

Let's look at one of the users, say the one with a userId == 97088.

In [ ]:
userSubsetGroup.get_group(97088)

Let's also sort these groups so the users that share the most movies in common with the input have higher priority. This provides a richer recommendation since we won't go through every single user.

In [ ]:
#Sorting it so users with movie most in common with the input will have priority
userSubsetGroup = sorted(userSubsetGroup,  key=lambda x: len(x[1]), reverse=True)

Let's look at the first 3 users.

In [ ]:
userSubsetGroup[0:3]

#### Similarity of users to input user
Next, we are going to compare all users (not really all !!!) to our specified user and find the one that is most similar.  
we're going to find out how similar each user is to the input through the __Pearson Correlation Coefficient__. It is used to measure the strength of a linear association between two variables. The formula for finding this coefficient between sets X and Y with N values can be seen in the image below. 

Why Pearson Correlation?

Pearson correlation is invariant to scaling, i.e. multiplying all elements by a nonzero constant or adding any constant to all elements. For example, if you have two vectors X and Y,then, pearson(X, Y) == pearson(X, 2 * Y + 3). This is a pretty important property in recommendation systems because for example two users might rate two series of items totally different in terms of absolute rates, but they would be similar users (i.e. with similar ideas) with similar rates in various scales .

![alt text](https://wikimedia.org/api/rest_v1/media/math/render/svg/bd1ccc2979b0fd1c1aec96e386f686ae874f9ec0 "Pearson Correlation")

The values given by the formula vary from r = -1 to r = 1, where 1 forms a direct correlation between the two entities (it means a perfect positive correlation) and -1 forms a perfect negative correlation. 

In our case, a 1 means that the two users have similar tastes while a -1 means the opposite.

We will select a subset of users to iterate through. This limit is imposed because we don't want to waste too much time going through every single user.

In [ ]:
userSubsetGroup = userSubsetGroup[0:100]

Now, we calculate the Pearson Correlation between input user and subset group, and store it in a dictionary, where the key is the user Id and the value is the coefficient

In [ ]:
from math import sqrt

In [ ]:
#Store the Pearson Correlation in a dictionary, where the key is the user Id and the value is the coefficient
pearsonCorrelationDict = {}

#For every user group in our subset
for name, group in userSubsetGroup:
    #Let's start by sorting the input and current user group so the values aren't mixed up later on
    group = group.sort_values(by='movieId')
    inputMovies = inputMovies.sort_values(by='movieId')
    #Get the N for the formula
    nRatings = len(group)
    #Get the review scores for the movies that they both have in common
    temp_df = inputMovies[inputMovies['movieId'].isin(group['movieId'].tolist())]
    #And then store them in a temporary buffer variable in a list format to facilitate future calculations
    tempRatingList = temp_df['rating'].tolist()
    #Let's also put the current user group reviews in a list format
    tempGroupList = group['rating'].tolist()
    #Now let's calculate the pearson correlation between two users, so called, x and y
    Sxx = sum([i**2 for i in tempRatingList]) - pow(sum(tempRatingList),2)/float(nRatings)
    Syy = sum([i**2 for i in tempGroupList]) - pow(sum(tempGroupList),2)/float(nRatings)
    Sxy = sum( i*j for i, j in zip(tempRatingList, tempGroupList)) - sum(tempRatingList)*sum(tempGroupList)/float(nRatings)
    
    #If the denominator is different than zero, then divide, else, 0 correlation.
    if Sxx != 0 and Syy != 0:
        pearsonCorrelationDict[name] = Sxy/sqrt(Sxx*Syy)
    else:
        pearsonCorrelationDict[name] = 0

In [ ]:
pearsonCorrelationDict.items()

As we can see, this dictionary is an eyesore and it is better to have the values stored in a dataframe.

In [ ]:
pearsonDF = pd.DataFrame.from_dict(pearsonCorrelationDict, orient='index')
pearsonDF.columns = ['similarityIndex']
pearsonDF['userId'] = pearsonDF.index
pearsonDF.index = range(len(pearsonDF))
pearsonDF.head()

#### The top x similar users to input user
Now let's get the top 50 users that are most similar to the input.

In [ ]:
topUsers=pearsonDF.sort_values(by='similarityIndex', ascending=False)[0:50]
topUsers.head()

Great, now then let's start recommending some movies to the input user.

#### Rating of selected users to all movies
We're going to do this by taking the weighted average of the ratings of the movies using the Pearson Correlation as the weight. But to do this, we first need to get the movies watched by the users in our __pearsonDF__ from the train dataframe and then store their correlation in a new column called _similarityIndex". This is achieved below by merging of these two tables.

In [ ]:
topUsersRating=topUsers.merge(df_train, left_on='userId', right_on='userId', how='inner')
topUsersRating.head()

Now all we need to do is simply multiply the movie rating by its weight (The similarity index), then sum up the new ratings and divide it by the sum of the weights.

We can easily do this by simply multiplying two columns, then grouping up the dataframe by movieId and then dividing two columns:

It shows the idea of all similar users to candidate movies for the input user:

In [ ]:
#Multiplies the similarity by the user's ratings
topUsersRating['weightedRating'] = topUsersRating['similarityIndex']*topUsersRating['rating']
topUsersRating.head()

In [ ]:
#Applies a sum to the topUsers after grouping it up by userId
tempTopUsersRating = topUsersRating.groupby('movieId').sum()[['similarityIndex','weightedRating']]
tempTopUsersRating.columns = ['sum_similarityIndex','sum_weightedRating']
tempTopUsersRating.head()

In [ ]:
#Creates an empty dataframe
recommendation_df = pd.DataFrame()
#Now we take the weighted average
recommendation_df['weighted average recommendation score'] = tempTopUsersRating['sum_weightedRating']/tempTopUsersRating['sum_similarityIndex']
recommendation_df['movieId'] = tempTopUsersRating.index
recommendation_df.head()

Now let's sort the recommendation dataframe and see just the top 20 movies that the algorithm recommended. 

In [ ]:
recommendation_df = recommendation_df.sort_values(by='weighted average recommendation score', ascending=False)
recommendation_df.head(10)

We need to make the recommended movies more recognizable.

In [ ]:
df_movies.loc[df_movies['movieId'].isin(recommendation_df.head(10)['movieId'].tolist())]

# Modelling

In [ ]:
from surprise import Reader, Dataset
reader = Reader()
data = Dataset.load_from_df(df_train[['userId', 'movieId', 'rating']], reader)

In [ ]:
from surprise.model_selection import train_test_split

trainset, testset = train_test_split(data, test_size=0.25)

### SVD

The Singular Value Decomposition (SVD), a method from linear algebra that has been generally used as a dimensionality reduction technique in machine learning. SVD is a matrix factorisation technique, which reduces the number of features of a dataset by reducing the space dimension from N-dimension to K-dimension (where K<N). In the context of the recommender system, the SVD is used as a collaborative filtering technique. It uses a matrix structure where each row represents a user, and each column represents an item. The elements of this matrix are the ratings that are given to items by users.

In [ ]:
from surprise import SVD, accuracy
algo = SVD()
algo.fit(trainset)

In [ ]:
predictions = algo.test(testset)


In [ ]:
# from surprise import SVD,Reader,Dataset

In [ ]:
# # Load Reader library
# reader = Reader()

# # Load ratings dataset with Dataset library
# data = Dataset.load_from_df(df_train[['userId', 'movieId', 'rating']], reader)

In [ ]:
# Use the SVD algorithm.
svd = SVD()

# Compute the RMSE of the SVD algorithm.
#cross_evaluate(svd, data, measures=['RMSE'])

In [ ]:
# cross_validate(svd, data, measures=['RMSE'], cv=3, verbose=True)

We get a mean Root Mean Square Error of 0.85 which is pretty good. Let's now train on the dataset and arrive at predictions.

In [ ]:
# trainset = data.build_full_trainset()
# svd.fit(trainset)

Now let's use SVD to predict the rating that, for instance, a User with ID 150 will give to a random movie (let's say with Movie ID 1000).

In [ ]:
# svd.predict(150, 1000)

For a movie with movieId 1000, User with ID 150 is predicted to give a rating of 3.3.

### SVD++

In [ ]:
# from surprise.model_selection import train_test_split

# trainset, testset = train_test_split(data, test_size=0.25)

In [ ]:
# algo_svdpp = SVDpp(n_factors=160, n_epochs=10, lr_all=0.005, reg_all=0.1)
# algo_svdpp.fit(trainset)
# test_pred = algo_svdpp.test(testset)

### Baseline Only

A baseline is a method that uses heuristics, simple summary statistics, randomness, or machine learning to create predictions for a dataset.The purpose of using a baseline when evaluating a machine learning model algorithm is to provides a set of predictions that you can evaluate as you would any predictions for your problem, such as classification accuracy or RMSE

In [ ]:
bsl_options = {'method': 'sgd','n_epochs': 35}
blo = BaselineOnly(bsl_options=bsl_options)
blo.fit(trainset)
predictions = blo.test(testset)
blo_rmse = accuracy.rmse(predictions)

### SlopeOne

Slope one algorithm is a special form of Item-based collaborative filtering algorithm, which has the characteristics of easy to use, accurate recommendation and high computational efficiency.

In [ ]:
# SlopeOne
slo_test = SlopeOne()
slo_test.fit(trainset)
predictions = slo_test.test(testset)
# Calculate RMSE
slo_rmse = accuracy.rmse(predictions)

### NMF

Non-Negative Matrix Factorization (NMF) is an unsupervised algorithm that projects data into lower dimensional spaces, effectively reducing the number of features while retaining the basis information necessary to reconstruct the original data

In [ ]:
nmf = NMF()
nmf.fit(trainset)
nmf_p = nmf.test(testset)
nmf_rmse = accuracy.rmse(nmf_p)

### Normal Predictor

NormalPredictor algorithm predicts a random rating based on the distribution of the training set, which is assumed to be normal.

In [ ]:
np = NormalPredictor()
np.fit(trainset)
np_p = np.test(testset)
np_rmse = accuracy.rmse(np_p)

### CoClustering

Co-clustering simultaneously cluster both data points and features of a two-dimensional data matrix. It is a powerful data analysis technique that can discover latent patterns hidden within particular rows and columns

In [ ]:
cc = CoClustering(random_state=42)
cc.fit(trainset)
cc_p = cc.test(testset)
cc_rmse = accuracy.rmse(cc_p)

# Model Evaluation

We built and tested different types of collaborative filtering models . Their performance was compared using a statistical measure known as the root mean squared error (RMSE), which determines the average squared difference between the estimated values and the actual value. A low RMSE value indicates high model accuracy.

### Root Mean Squared Error (RMSE):

$$RMSE = \sqrt{\frac{1}{|\hat{R}|}\Sigma_{r_{ui} \in {\hat{R}}}{\Big({r_{ui} -{\hat {r}}_{ui}}\Big)^2}}$$  

Where  𝑅̂   is the total number of recommendations generated for users and movies, with  𝑟𝑢𝑖  and  𝑟̂ 𝑢𝑖  being the true and predicted ratings for user u watching movie i respectively.

In [ ]:
# Compare RMSE Between Models
fig,axis = plt.subplots(figsize=(14, 5))
models = ['SVD','Normal Predictor','NMF','CoClustering','Baseline Only','SlopeOne']
rmse_y = [svd_rmse,np_rmse,nmf_rmse,cc_rmse,blo_rmse,slo_rmse]
ax = sns.barplot(x=models, y=rmse_y,palette='winter')
plt.title('RMSE Value Per Collaborative-based Filtering Model',fontsize=14)
plt.xticks(rotation=90)
plt.ylabel('RMSE')
for p in ax.patches:
    ax.text(p.get_x() + p.get_width()/2, p.get_y() + p.get_height(), round(p.get_height(),4), fontsize=12, ha="center", va='bottom')
    
plt.show()

### Submission

In [ ]:
# test predictions
predictions = []
for i, row in df_test.iterrows():
    x = (algo.predict(row.userId, row.movieId))
    pred = x[3]
    predictions.append(pred)

In [ ]:
df_test['Id'] =df_test['userId'].map(str) +'_'+ df_test['movieId'].map(str)
results = pd.DataFrame({"Id":df_test['Id'],"rating": predictions})
results.to_csv("Team JS1-Unsupervised.csv", index=False)